In [1]:
import nltk
import os
#nltk.download() 
#布朗语料库，如果使用预训练模型，则不需要
# Generate brown corpus text file
with open('brown_corp.txt', 'w+') as f:
    for word in nltk.corpus.brown.words():
        f.write('{word} '.format(word=word))   

/usr/bin/sh: /home/tf/workspace/anxingle/research/fastText/nltk_data: Is a directory


In [9]:
# 下载text8 语料库 (a 100 MB sample of cleaned wikipedia text)
!wget http://mattmahoney.net/dc/text8.zip

In [8]:
# 下载问答语料库, 测试用！ questions-words.txt to be used for comparing word embeddings
!wget https://raw.githubusercontent.com/nicholas-leonard/word2vec/master/questions-words.txt

## 训练fastText模型

In [12]:
!../fasttext skipgram -input ./brown_corp.txt -output brown_ft_by_anxingle
!../fasttext skipgram -input ./text8 -output text8_ft_by_anxingle

Read 1M words
Number of words:  15173
Number of labels: 0
Progress: 100.0%  words/sec/thread: 64536  lr: 0.000000  loss: 2.364343  eta: 0h0m  2.388624  eta: 0h0m 0h0m 0.014048  loss: 2.355604  eta: 0h0m 0.012411  loss: 2.355630  eta: 0h0m   eta: 0h0m 
Read 17M words
Number of words:  71290
Number of labels: 0
Progress: 100.0%  words/sec/thread: 58843  lr: 0.000000  loss: 1.762129  eta: 0h0m  loss: 2.027483  eta: 0h3m 36200  lr: 0.049568  loss: 2.005225  eta: 0h3m h3m 1.840394  eta: 0h2m 1.835855  eta: 0h1m 0.042200  loss: 1.839727  eta: 0h1m   lr: 0.042057  loss: 1.839946  eta: 0h1m m   eta: 0h1m 0.039760  loss: 1.837245  eta: 0h1m 1.836794  eta: 0h1m 57230  lr: 0.039395  loss: 1.836822  eta: 0h1m 0h1m 0.036827  loss: 1.833853  eta: 0h1m   loss: 1.834084  eta: 0h1m 1.834594  eta: 0h1m   words/sec/thread: 57724  lr: 0.034135  loss: 1.833861  eta: 0h1m 1.833246  eta: 0h1m   lr: 0.032758  loss: 1.825619  eta: 0h1m   lr: 0.032686  loss: 1.825747  eta: 0h1m   eta: 0h1m 0.032370  loss: 1.825

## 训练gensim（word2vec）模型

In [18]:
from nltk.corpus import brown
from gensim.models import Word2Vec
from gensim.models.word2vec import Text8Corpus
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)

MODELS_DIR = 'models/'

brown_gs = Word2Vec(brown.sents())
brown_gs.wv.save_word2vec_format(MODELS_DIR + 'brown_gs.vec')

text8_gs = Word2Vec(Text8Corpus('text8'))
text8_gs.wv.save_word2vec_format(MODELS_DIR + 'text8_gs.vec')
#model.wv.save_word2vec_format model.wv.save_word2vec_format 


"\nfrom nltk.corpus import brown\nfrom gensim.models import Word2Vec\nfrom gensim.models.word2vec import Text8Corpus\nimport logging\n\nlogging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')\nlogging.root.setLevel(level=logging.INFO)\n\nMODELS_DIR = 'models/'\n\nbrown_gs = Word2Vec(brown.sents())\nbrown_gs.wv.save_word2vec_format(MODELS_DIR + 'brown_gs.vec')\n\ntext8_gs = Word2Vec(Text8Corpus('text8'))\ntext8_gs.wv.save_word2vec_format(MODELS_DIR + 'text8_gs.vec')\n#model.wv.save_word2vec_format model.wv.save_word2vec_format \n"

## 模型（语义，句法）相似度度量

In [49]:
from gensim.models import Word2Vec
from gensim import models

def print_accuracy(model, questions_file):
    print('Evaluating...\n')
    acc = model.accuracy(questions_file)
    for section in acc:
        correct = len(section['correct'])
        total = len(section['correct']) + len(section['incorrect'])
        total = total if total else 1
        accuracy = 100*float(correct)/total
        print('{:d}/{:d}, {:.2f}%, Section: {:s}'.format(correct, total, accuracy, section['section']))
    sem_correct = sum((len(acc[i]['correct']) for i in range(5)))
    sem_total = sum((len(acc[i]['correct']) + len(acc[i]['incorrect'])) for i in range(5))
    print('\nSemantic: {:d}/{:d}, Accuracy: {:.2f}%'.format(sem_correct, sem_total, 100*float(sem_correct)/sem_total))
    
    syn_correct = sum((len(acc[i]['correct']) for i in range(5, len(acc)-1)))
    syn_total = sum((len(acc[i]['correct']) + len(acc[i]['incorrect'])) for i in range(5,len(acc)-1))
    print('Syntactic: {:d}/{:d}, Accuracy: {:.2f}%\n'.format(syn_correct, syn_total, 100*float(syn_correct)/syn_total))

## 测试

####  fastText模型在brown上的精度

In [50]:
from gensim.models import *
from gensim.models import Word2Vec
MODELS_DIR = "models/"
word_analogies_file = './questions-words.txt'
print('\nLoading FastText embeddings')
ft_model = KeyedVectors.load_word2vec_format(MODELS_DIR + 'brown_ft_by_anxingle.vec')
pp = ft_model.accuracy(word_analogies_file)
print('Accuracy for FastText:')
print_accuracy(ft_model, word_analogies_file)

2017-06-20 16:26:00,159 : INFO : loading projection weights from models/brown_ft_by_anxingle.vec


['.ipynb_checkpoints', 'tutorial.ipynb', 'brown_corp.txt', 'text8.zip', 'questions-words.txt', 'text8', 'models']

Loading FastText embeddings


2017-06-20 16:26:02,136 : INFO : loaded (15173, 100) matrix from models/brown_ft_by_anxingle.vec
2017-06-20 16:26:02,155 : INFO : precomputing L2-norms of word weight vectors
2017-06-20 16:26:02,359 : INFO : capital-common-countries: 1.1% (1/90)
2017-06-20 16:26:02,534 : INFO : capital-world: 0.0% (0/44)
2017-06-20 16:26:02,571 : INFO : currency: 0.0% (0/12)
2017-06-20 16:26:03,640 : INFO : city-in-state: 2.6% (12/457)
2017-06-20 16:26:04,023 : INFO : family: 17.6% (37/210)
2017-06-20 16:26:05,459 : INFO : gram1-adjective-to-adverb: 66.5% (503/756)
2017-06-20 16:26:05,711 : INFO : gram2-opposite: 78.0% (103/132)
2017-06-20 16:26:07,845 : INFO : gram3-comparative: 59.3% (626/1056)
2017-06-20 16:26:08,233 : INFO : gram4-superlative: 77.1% (162/210)
2017-06-20 16:26:09,494 : INFO : gram5-present-participle: 64.8% (421/650)
2017-06-20 16:26:10,060 : INFO : gram6-nationality-adjective: 32.7% (97/297)
2017-06-20 16:26:12,568 : INFO : gram7-past-tense: 11.7% (147/1260)
2017-06-20 16:26:13,761

Accuracy for FastText:
Evaluating...



2017-06-20 16:26:14,676 : INFO : capital-common-countries: 1.1% (1/90)
2017-06-20 16:26:14,813 : INFO : capital-world: 0.0% (0/44)
2017-06-20 16:26:14,909 : INFO : currency: 0.0% (0/12)
2017-06-20 16:26:15,884 : INFO : city-in-state: 2.6% (12/457)
2017-06-20 16:26:16,371 : INFO : family: 17.6% (37/210)
2017-06-20 16:26:17,915 : INFO : gram1-adjective-to-adverb: 66.5% (503/756)
2017-06-20 16:26:18,162 : INFO : gram2-opposite: 78.0% (103/132)
2017-06-20 16:26:20,129 : INFO : gram3-comparative: 59.3% (626/1056)
2017-06-20 16:26:20,580 : INFO : gram4-superlative: 77.1% (162/210)
2017-06-20 16:26:21,802 : INFO : gram5-present-participle: 64.8% (421/650)
2017-06-20 16:26:22,434 : INFO : gram6-nationality-adjective: 32.7% (97/297)
2017-06-20 16:26:24,803 : INFO : gram7-past-tense: 11.7% (147/1260)
2017-06-20 16:26:25,851 : INFO : gram8-plural: 56.7% (313/552)
2017-06-20 16:26:26,537 : INFO : gram9-plural-verbs: 71.6% (245/342)
2017-06-20 16:26:26,539 : INFO : total: 44.0% (2667/6068)


1/90, 1.11%, Section: capital-common-countries
0/44, 0.00%, Section: capital-world
0/12, 0.00%, Section: currency
12/457, 2.63%, Section: city-in-state
37/210, 17.62%, Section: family
503/756, 66.53%, Section: gram1-adjective-to-adverb
103/132, 78.03%, Section: gram2-opposite
626/1056, 59.28%, Section: gram3-comparative
162/210, 77.14%, Section: gram4-superlative
421/650, 64.77%, Section: gram5-present-participle
97/297, 32.66%, Section: gram6-nationality-adjective
147/1260, 11.67%, Section: gram7-past-tense
313/552, 56.70%, Section: gram8-plural
245/342, 71.64%, Section: gram9-plural-verbs
2667/6068, 43.95%, Section: total

Semantic: 50/813, Accuracy: 6.15%
Syntactic: 2617/5255, Accuracy: 49.80%



####  word2vec模型在brown上的精度

In [52]:
print('\nLoading Gensim embeddings')
gs_model = KeyedVectors.load_word2vec_format(MODELS_DIR + 'brown_gs.vec')
print('Accuracy for word2vec:')
print_accuracy(gs_model, word_analogies_file)

2017-06-20 16:27:44,567 : INFO : loading projection weights from models/brown_gs.vec



Loading Gensim embeddings


2017-06-20 16:27:46,548 : INFO : loaded (15173, 100) matrix from models/brown_gs.vec
2017-06-20 16:27:46,590 : INFO : precomputing L2-norms of word weight vectors


Accuracy for word2vec:
Evaluating...



2017-06-20 16:27:46,835 : INFO : capital-common-countries: 0.0% (0/90)
2017-06-20 16:27:46,970 : INFO : capital-world: 0.0% (0/44)
2017-06-20 16:27:47,003 : INFO : currency: 0.0% (0/12)
2017-06-20 16:27:47,855 : INFO : city-in-state: 0.0% (0/457)
2017-06-20 16:27:48,270 : INFO : family: 28.1% (59/210)
2017-06-20 16:27:49,739 : INFO : gram1-adjective-to-adverb: 0.7% (5/756)
2017-06-20 16:27:49,976 : INFO : gram2-opposite: 0.0% (0/132)
2017-06-20 16:27:52,109 : INFO : gram3-comparative: 6.8% (72/1056)
2017-06-20 16:27:52,506 : INFO : gram4-superlative: 0.0% (0/210)
2017-06-20 16:27:53,754 : INFO : gram5-present-participle: 2.2% (14/650)
2017-06-20 16:27:54,303 : INFO : gram6-nationality-adjective: 0.0% (0/297)
2017-06-20 16:27:56,950 : INFO : gram7-past-tense: 2.3% (29/1260)
2017-06-20 16:27:57,943 : INFO : gram8-plural: 0.7% (4/552)
2017-06-20 16:27:58,566 : INFO : gram9-plural-verbs: 2.3% (8/342)
2017-06-20 16:27:58,568 : INFO : total: 3.1% (191/6068)


0/90, 0.00%, Section: capital-common-countries
0/44, 0.00%, Section: capital-world
0/12, 0.00%, Section: currency
0/457, 0.00%, Section: city-in-state
59/210, 28.10%, Section: family
5/756, 0.66%, Section: gram1-adjective-to-adverb
0/132, 0.00%, Section: gram2-opposite
72/1056, 6.82%, Section: gram3-comparative
0/210, 0.00%, Section: gram4-superlative
14/650, 2.15%, Section: gram5-present-participle
0/297, 0.00%, Section: gram6-nationality-adjective
29/1260, 2.30%, Section: gram7-past-tense
4/552, 0.72%, Section: gram8-plural
8/342, 2.34%, Section: gram9-plural-verbs
191/6068, 3.15%, Section: total

Semantic: 59/813, Accuracy: 7.26%
Syntactic: 132/5255, Accuracy: 2.51%



## text8语料库

#### fastText模型在text8语料库上的精度

In [58]:
print('\nLoading FastText embeddings')
word_analogies_file = './questions-words.txt'
ft_model = KeyedVectors.load_word2vec_format(MODELS_DIR + 'text8_ft_by_anxingle.vec')
pp = ft_model.accuracy(word_analogies_file)
print('Accuracy for FastText:')
print_accuracy(ft_model, word_analogies_file)

2017-06-20 16:49:57,652 : INFO : loading projection weights from models/text8_ft_by_anxingle.vec



Loading FastText embeddings


2017-06-20 16:50:06,249 : INFO : loaded (71290, 100) matrix from models/text8_ft_by_anxingle.vec
2017-06-20 16:50:06,285 : INFO : precomputing L2-norms of word weight vectors
2017-06-20 16:50:08,327 : INFO : capital-common-countries: 61.7% (312/506)
2017-06-20 16:50:13,934 : INFO : capital-world: 41.4% (601/1452)
2017-06-20 16:50:14,968 : INFO : currency: 16.0% (43/268)
2017-06-20 16:50:20,841 : INFO : city-in-state: 16.5% (250/1511)
2017-06-20 16:50:22,068 : INFO : family: 47.4% (145/306)
2017-06-20 16:50:24,917 : INFO : gram1-adjective-to-adverb: 71.6% (541/756)
2017-06-20 16:50:26,186 : INFO : gram2-opposite: 60.8% (186/306)
2017-06-20 16:50:31,047 : INFO : gram3-comparative: 67.6% (852/1260)
2017-06-20 16:50:33,030 : INFO : gram4-superlative: 55.7% (282/506)
2017-06-20 16:50:36,696 : INFO : gram5-present-participle: 54.6% (542/992)
2017-06-20 16:50:41,580 : INFO : gram6-nationality-adjective: 94.4% (1294/1371)
2017-06-20 16:50:46,075 : INFO : gram7-past-tense: 33.8% (450/1332)
2017

Accuracy for FastText:
Evaluating...



2017-06-20 16:50:53,909 : INFO : capital-common-countries: 61.7% (312/506)
2017-06-20 16:50:59,284 : INFO : capital-world: 41.4% (601/1452)
2017-06-20 16:51:00,254 : INFO : currency: 16.0% (43/268)
2017-06-20 16:51:05,645 : INFO : city-in-state: 16.5% (250/1511)
2017-06-20 16:51:06,731 : INFO : family: 47.4% (145/306)
2017-06-20 16:51:09,689 : INFO : gram1-adjective-to-adverb: 71.6% (541/756)
2017-06-20 16:51:10,864 : INFO : gram2-opposite: 60.8% (186/306)
2017-06-20 16:51:15,764 : INFO : gram3-comparative: 67.6% (852/1260)
2017-06-20 16:51:17,840 : INFO : gram4-superlative: 55.7% (282/506)
2017-06-20 16:51:21,565 : INFO : gram5-present-participle: 54.6% (542/992)
2017-06-20 16:51:26,879 : INFO : gram6-nationality-adjective: 94.4% (1294/1371)
2017-06-20 16:51:32,035 : INFO : gram7-past-tense: 33.8% (450/1332)
2017-06-20 16:51:35,822 : INFO : gram8-plural: 87.9% (872/992)
2017-06-20 16:51:38,400 : INFO : gram9-plural-verbs: 58.9% (383/650)
2017-06-20 16:51:38,402 : INFO : total: 55.3% (

312/506, 61.66%, Section: capital-common-countries
601/1452, 41.39%, Section: capital-world
43/268, 16.04%, Section: currency
250/1511, 16.55%, Section: city-in-state
145/306, 47.39%, Section: family
541/756, 71.56%, Section: gram1-adjective-to-adverb
186/306, 60.78%, Section: gram2-opposite
852/1260, 67.62%, Section: gram3-comparative
282/506, 55.73%, Section: gram4-superlative
542/992, 54.64%, Section: gram5-present-participle
1294/1371, 94.38%, Section: gram6-nationality-adjective
450/1332, 33.78%, Section: gram7-past-tense
872/992, 87.90%, Section: gram8-plural
383/650, 58.92%, Section: gram9-plural-verbs
6753/12208, 55.32%, Section: total

Semantic: 1351/4043, Accuracy: 33.42%
Syntactic: 5402/8165, Accuracy: 66.16%



#### word2vec在text8上的精度

In [60]:
print('\nLoading word2vec embeddings')
ft_model = KeyedVectors.load_word2vec_format(MODELS_DIR + 'text8_gs.vec')
pp = ft_model.accuracy(word_analogies_file)
print('Accuracy for FastText:')
print_accuracy(ft_model, word_analogies_file)

2017-06-20 16:51:48,220 : INFO : loading projection weights from models/text8_gs.vec



Loading word2vec embeddings


2017-06-20 16:51:57,509 : INFO : loaded (71290, 100) matrix from models/text8_gs.vec
2017-06-20 16:51:57,574 : INFO : precomputing L2-norms of word weight vectors
2017-06-20 16:51:59,438 : INFO : capital-common-countries: 26.7% (135/506)
2017-06-20 16:52:05,241 : INFO : capital-world: 16.9% (245/1452)
2017-06-20 16:52:06,283 : INFO : currency: 9.3% (25/268)
2017-06-20 16:52:12,544 : INFO : city-in-state: 8.8% (139/1571)
2017-06-20 16:52:13,643 : INFO : family: 75.8% (232/306)
2017-06-20 16:52:16,669 : INFO : gram1-adjective-to-adverb: 11.6% (88/756)
2017-06-20 16:52:17,884 : INFO : gram2-opposite: 14.7% (45/306)
2017-06-20 16:52:22,815 : INFO : gram3-comparative: 56.3% (709/1260)
2017-06-20 16:52:24,777 : INFO : gram4-superlative: 37.5% (190/506)
2017-06-20 16:52:28,598 : INFO : gram5-present-participle: 30.7% (305/992)
2017-06-20 16:52:33,983 : INFO : gram6-nationality-adjective: 49.5% (679/1371)
2017-06-20 16:52:39,184 : INFO : gram7-past-tense: 25.8% (344/1332)
2017-06-20 16:52:43,0

Accuracy for FastText:
Evaluating...



2017-06-20 16:52:47,683 : INFO : capital-common-countries: 26.7% (135/506)
2017-06-20 16:52:53,311 : INFO : capital-world: 16.9% (245/1452)
2017-06-20 16:52:54,340 : INFO : currency: 9.3% (25/268)
2017-06-20 16:53:00,472 : INFO : city-in-state: 8.8% (139/1571)
2017-06-20 16:53:01,643 : INFO : family: 75.8% (232/306)
2017-06-20 16:53:04,571 : INFO : gram1-adjective-to-adverb: 11.6% (88/756)
2017-06-20 16:53:05,763 : INFO : gram2-opposite: 14.7% (45/306)
2017-06-20 16:53:10,758 : INFO : gram3-comparative: 56.3% (709/1260)
2017-06-20 16:53:12,663 : INFO : gram4-superlative: 37.5% (190/506)
2017-06-20 16:53:16,499 : INFO : gram5-present-participle: 30.7% (305/992)
2017-06-20 16:53:21,788 : INFO : gram6-nationality-adjective: 49.5% (679/1371)
2017-06-20 16:53:26,986 : INFO : gram7-past-tense: 25.8% (344/1332)
2017-06-20 16:53:30,789 : INFO : gram8-plural: 38.1% (378/992)
2017-06-20 16:53:33,274 : INFO : gram9-plural-verbs: 31.4% (204/650)
2017-06-20 16:53:33,276 : INFO : total: 30.3% (3718/

135/506, 26.68%, Section: capital-common-countries
245/1452, 16.87%, Section: capital-world
25/268, 9.33%, Section: currency
139/1571, 8.85%, Section: city-in-state
232/306, 75.82%, Section: family
88/756, 11.64%, Section: gram1-adjective-to-adverb
45/306, 14.71%, Section: gram2-opposite
709/1260, 56.27%, Section: gram3-comparative
190/506, 37.55%, Section: gram4-superlative
305/992, 30.75%, Section: gram5-present-participle
679/1371, 49.53%, Section: gram6-nationality-adjective
344/1332, 25.83%, Section: gram7-past-tense
378/992, 38.10%, Section: gram8-plural
204/650, 31.38%, Section: gram9-plural-verbs
3718/12268, 30.31%, Section: total

Semantic: 776/4103, Accuracy: 18.91%
Syntactic: 2942/8165, Accuracy: 36.03%

